v1.0과 v1.1의 차이점은,
v1.1에서는 pickle 파일을 데이터로 불러온다.

In [2]:
import os
import pandas as pd
import numpy as np
os.getcwd()
import openpyxl

import pickle
from tqdm.notebook import tnrange, tqdm

# 1. train, test set을 로딩합니다.

In [4]:
# train pickle 파일 Loading
with open('/home/hb/python/pp_pred/from_kw/ignore_file/dfpsphmm_ratiorandomver2.pickle', 'rb') as file:
  df_train = pickle.load(file)

df_train = pd.DataFrame(df_train,columns=['GENE','KINASE','KIN_ACC_ID','KIN_ORGANISM','SUBSTRATE',
                                'SUB_GENE_ID','SUB_ACC_ID','SUB_GENE','SUB_ORGANISM','SUB_MOD_RSD','SITE_GRP_ID',
                                'SITE_+/-7_AA','DOMAIN','IN_VIVO_RXN','IN_VITRO_RXN','CST_CAT#','KINseq',
                                'Length','answer','singleSITE','matrix'])

# test pickle 파일 loading
with open( "/home/hb/python/pp_pred/from_kw/ignore_file/dfdepodhmm_ratiorandomver2.pickle", "rb" ) as file:
    df_test = pickle.load(file)
df_test.rename(columns = {'kinseq' : 'KINseq','Substrate' : 'SITE_+/-7_AA'}, inplace = True)
# df_test = df_test[['KINseq','SITE_+/-7_AA','matrix']]
# df_test = df7[:445]

display(df_train.head(3))
display(df_test.head(3))

,GENE,KINASE,KIN_ACC_ID,KIN_ORGANISM,SUBSTRATE,SUB_GENE_ID,SUB_ACC_ID,SUB_GENE,SUB_ORGANISM,SUB_MOD_RSD,...,SITE_+/-7_AA,DOMAIN,IN_VIVO_RXN,IN_VITRO_RXN,CST_CAT#,KINseq,Length,answer,singleSITE,matrix
0,Dyrk2,DYRK2,Q5U4C9,mouse,NDEL1,83431.0,Q9ERR1,Ndel1,mouse,S336,...,LGSSRPSSAPGMLPL,NaN,,X,NaN,YEVLKVIGKGSFGQVVKAYDHKVHQHVALKMVRNEKRFHRQAAEEI...,313,1,"[L, G, S, S, R, P, S, S, A, P, G, M, L, P, L]","[[[1.152, 0.981, 1.023, 1.023, 1.062, 0.886, 1..."
1,Pak2,PAK2,Q64303,rat,MEK1,170851.0,Q01986,Map2k1,rat,S298,...,RTPGRPLSSYGMDSR,Pkinase,,X,9128; 98195,YTRYEKIGQGASGTVFTATDVALGQEVAIKQINLQKQPKKELIINE...,251,1,"[R, T, P, G, R, P, L, S, S, Y, G, M, D, S, R]","[[[1.062, 1.027, 0.886, 0.981, 1.062, 0.886, 1..."
2,Pak2,PAK2,Q64303,rat,PRKD1,85421.0,Q9WTQ1,Prkd1,rat,S203,...,GVRRRRLSNVSLTGL,NaN,X,,NaN,YTRYEKIGQGASGTVFTATDVALGQEVAIKQINLQKQPKKELIINE...,251,1,"[G, V, R, R, R, R, L, S, N, V, S, L, T, G, L]","[[[0.981, 1.165, 1.062, 1.062, 1.062, 1.062, 1..."


,Substrate accession number,Substrate Gene name/s,SITE_+/-7_AA,Phosphorylating protein kinases,Bioassay/s,Literature support,Source database,Dephosphorylation site and 5 amino acid window around it,Dephosphorylating protein phosphatases,Literature support.1,KINseq,Length,answer,matrix
4,O00401,"N-WASP, WASL",EITTNRFYGPQVNNI,ABL1,"in vitro, in vivo","16199863, 17389395, 18452278, 18707149, 194156...",HPRD,y256_TSKVIyDFIEK,PTPN2,16293614,MKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAV...,247,1,"[[[0.965, 1.207, 1.078, 1.078, 0.9233, 0.993, ..."
4,O00401,"N-WASP, WASL",RETSKVIYDFIEKTG,ABL1,"in vitro, in vivo","16199863, 17389395, 18452278, 18707149, 194156...",HPRD,y256_TSKVIyDFIEK,PTPN2,16293614,MKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAV...,247,1,"[[[0.993, 0.965, 1.078, 1.0, 0.886, 1.229, 1.2..."
21,O14745,"NHERF1, SLC9A3R1",PALVRSASSDTSEEL,GRK6,in vivo,"15302935, 18088087, 18452278, 18669648, 194133...",HPRD,"340_340, s339_AHQKRSSKRAP HQKRSsKRAPQ","PPP2CA, PPP2CB, PTP4A3","22301917,22523604, 25897829",RQYRVLGKGGFGEVCACQVRATGKMYACKKLEKKRIKKRKGEAMAL...,261,1,"[[[0.7417, 1.137, 1.097, 1.035, 0.98, 0.881, 1..."


# 2. train, test data preprocessing
- train data에 human외 다른 종의 데이터도 포함되어 있기 때문에, human 데이터만 뽑아줍니다.
- 필요한 컬럼만 뽑아줍니다.  columns = 'sub_ID', 'sub_name', 'sub_seq', 'kin_name', 'kin_seq', 'len', 'answer', 'matrix'
    - 'sub_ID' : substrate의 Uniprot ID
    - 'sub_name' : substrate의 name
    - 'sub_seq' : substrate의 sequence(SITE_+/-7_AA	 : phosphosite)
    - 'kin_name' : kinase의 name
    - 'kin_seq' : kinase의 sequence(HMM적용 후)
    - 'len' : kinase sequence의 길이
    - 'answer' : binding여부 - binding O(1) / binding X(0)
    - 'matrix' : input data로 들어가는 matrix


In [5]:
# train data에는 human외에 다른 종의 데이터도 포함되어 있기 때문에, human 데이터만 뽑아준다.

df_train_kin_human = df_train[df_train['KIN_ORGANISM'] == 'human']
df_train_human = df_train_kin_human[df_train_kin_human['SUB_ORGANISM'] == 'human']
df_train = df_train_human 

In [6]:
# 필요한 컬럼만 뽑아준다.
columns = 'sub_ID', 'sub_name', 'sub_seq', 'kin_name', 'kin_seq', 'len', 'answer', 'matrix'
df_train.columns
df_train = df_train.loc[:,['SUB_ACC_ID','SUBSTRATE','SITE_+/-7_AA','KINASE','KINseq','Length','answer','matrix']]
df_train.columns = columns
df_train.head(1)

,sub_ID,sub_name,sub_seq,kin_name,kin_seq,len,answer,matrix
7,Q9UQL6,HDAC5,FPLRKTASEPNLKVR,PKCD,HKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVE...,251,1,"[[[1.061, 0.825, 1.012, 0.963, 0.6885, 1.026, ..."


In [7]:
df_test.head(1)
df_test = df_test.loc[:,['Substrate accession number','Substrate Gene name/s','SITE_+/-7_AA','Phosphorylating protein kinases','KINseq','Length','answer','matrix']]
df_test.head(1)
df_test.columns = ['sub_ID', 'sub_name', 'sub_seq', 'kin_name', 'kin_seq', 'len', 'answer', 'matrix']
df_test.head(1)

,sub_ID,sub_name,sub_seq,kin_name,kin_seq,len,answer,matrix
4,O00401,"N-WASP, WASL",EITTNRFYGPQVNNI,ABL1,MKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAV...,247,1,"[[[0.965, 1.207, 1.078, 1.078, 0.9233, 0.993, ..."


# 3. 중복데이터 처리(1) by matrix
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html
- pandas의 merge함수를 사용하여 중복데이터를 처리한다.
- merger함수의 how='inner' 방식은 left, right dataframe 모두에 있는 값만 추출한다. (즉, 교집합 = 중복데이터)
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FmUaDA%2FbtqKSfLmuhO%2FK3cMP0Znh9NtUoRTODNbZK%2Fimg.png">


# 4. 중복데이터 처리(2) by aa seq pair
## 4.1 merge 함수 이용
- substrate aa seq - kinase aa seq 로 해서 중복되는 데이터가 없는지 체크하기 위해  'subseq_kinseq'이란 열을 만들어서, 'Substrate_seq'와 'Kinase_seq'열의 값을 '_'로 연결해줍니다.
- 결과 : dup_seq


In [8]:
display(df_train.head(1))
display(df_test.head(1))

,sub_ID,sub_name,sub_seq,kin_name,kin_seq,len,answer,matrix
7,Q9UQL6,HDAC5,FPLRKTASEPNLKVR,PKCD,HKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVE...,251,1,"[[[1.061, 0.825, 1.012, 0.963, 0.6885, 1.026, ..."


,sub_ID,sub_name,sub_seq,kin_name,kin_seq,len,answer,matrix
4,O00401,"N-WASP, WASL",EITTNRFYGPQVNNI,ABL1,MKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAV...,247,1,"[[[0.965, 1.207, 1.078, 1.078, 0.9233, 0.993, ..."


In [10]:
def subseq_kinseq(df):
    df.loc[:,'subseq_kinseq'] = 0
    df['subseq_kinseq'] = df['sub_seq'].str.cat(df['kin_seq'], sep='_')


subseq_kinseq(df_train)
subseq_kinseq(df_test)
# subseq_kinseq(train_test_duplicate_comparison)

In [11]:
dup_seq = pd.merge(df_train, df_test, on="subseq_kinseq", how="inner")
dup_seq

,sub_ID_x,sub_name_x,sub_seq_x,kin_name_x,kin_seq_x,len_x,answer_x,matrix_x,subseq_kinseq,sub_ID_y,sub_name_y,sub_seq_y,kin_name_y,kin_seq_y,len_y,answer_y,matrix_y
0,P35611,ADD1,KKKFRTPSFLKKSKK,PKCD,HKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVE...,251,1,"[[[0.6885, 0.6885, 0.6885, 1.061, 0.963, 1.026...",KKKFRTPSFLKKSKK_HKVLGKGSFGKVLLGELKGRGEYFAIKALK...,P35612,"ADD2, beta-adducin",KKKFRTPSFLKKSKK,PRKCD,HKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVE...,251,1,"[[[0.6885, 0.6885, 0.6885, 1.061, 0.963, 1.026..."
1,P40763,STAT3,DPGSAAPYLKTKFIC,Src,EVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMK...,242,1,"[[[0.673, 0.652, 0.7866, 0.822, 1.012, 1.012, ...",DPGSAAPYLKTKFIC_EVKLGQGCFGEVWMGTWNGTTRVAIKTLKP...,P40763,STAT3,DPGSAAPYLKTKFIC,SRC,EVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMK...,242,1,"[[[0.673, 0.652, 0.7866, 0.822, 1.012, 1.012, ..."
2,Q9NRY4,GRF-1,RNEEENIYSVPHDST,Src,EVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMK...,242,1,"[[[1.17, 0.7827, 0.8345, 0.8345, 0.8345, 0.782...",RNEEENIYSVPHDST_EVKLGQGCFGEVWMGTWNGTTRVAIKTLKP...,Q9NRY4,"ARHGAP35, GRLF1, p190 RhoGAP",RNEEENIYSVPHDST,SRC,EVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMK...,242,1,"[[[1.17, 0.7827, 0.8345, 0.8345, 0.8345, 0.782..."
3,O14745,NHERF,PALVRSASSDTSEEL,GRK6,RQYRVLGKGGFGEVCACQVRATGKMYACKKLEKKRIKKRKGEAMAL...,261,1,"[[[0.7417, 1.137, 1.097, 1.035, 0.98, 0.881, 1...",PALVRSASSDTSEEL_RQYRVLGKGGFGEVCACQVRATGKMYACKK...,O14745,"NHERF1, SLC9A3R1",PALVRSASSDTSEEL,GRK6,RQYRVLGKGGFGEVCACQVRATGKMYACKKLEKKRIKKRKGEAMAL...,261,1,"[[[0.7417, 1.137, 1.097, 1.035, 0.98, 0.881, 1..."
4,Q15418,p90RSK,DHEKKAYSFCGTVEY,PDK1,KFGKILGEGSFSTVVLARELATSREYAIKILEKRHIIKENKVPYVT...,247,1,"[[[1.006, 0.6885, 1.199, 0.8145, 0.8145, 0.927...",DHEKKAYSFCGTVEY_KFGKILGEGSFSTVVLARELATSREYAIKI...,Q15418,"RPS6KA1, p90RSK1",DHEKKAYSFCGTVEY,PDPK1,KFGKILGEGSFSTVVLARELATSREYAIKILEKRHIIKENKVPYVT...,247,1,"[[[1.006, 0.6885, 1.199, 0.8145, 0.8145, 0.927..."
5,Q05655,PKCD,FGESRASTFCGTPDY,PDK1,KFGKILGEGSFSTVVLARELATSREYAIKILEKRHIIKENKVPYVT...,247,1,"[[[0.976, 0.836, 1.199, 0.837, 0.7046, 0.9277,...",FGESRASTFCGTPDY_KFGKILGEGSFSTVVLARELATSREYAIKI...,Q05655,"PKC-delta, PRKCD",FGESRASTFCGTPDY,PDPK1,KFGKILGEGSFSTVVLARELATSREYAIKILEKRHIIKENKVPYVT...,247,1,"[[[0.976, 0.836, 1.199, 0.837, 0.7046, 0.9277,..."
6,P07948,LYN,RVIEDNEYTAREGAK,LYN,LVKRLGAGQFGEVWMGYYNNSTKVAVKTLKPGTMSVQAFLEEANLM...,243,1,"[[[1.097, 1.238, 1.198, 0.984, 0.8447, 0.87, 0...",RVIEDNEYTAREGAK_LVKRLGAGQFGEVWMGYYNNSTKVAVKTLK...,P07948,LYN,RVIEDNEYTAREGAK,LYN,LVKRLGAGQFGEVWMGYYNNSTKVAVKTLKPGTMSVQAFLEEANLM...,243,1,"[[[1.097, 1.238, 1.198, 0.984, 0.8447, 0.87, 0..."


## 4.2 DUPLICATED함수 이용

In [12]:
df_train.loc[:,['df']] = 'train'
df_train

df_test.loc[:,['df']] = 'test'
df_test

,sub_ID,sub_name,sub_seq,kin_name,kin_seq,len,answer,matrix,subseq_kinseq,df
4,O00401,"N-WASP, WASL",EITTNRFYGPQVNNI,ABL1,MKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAV...,247,1,"[[[0.965, 1.207, 1.078, 1.078, 0.9233, 0.993, ...",EITTNRFYGPQVNNI_MKHKLGGGQYGEVYEGVWKKYSLTVAVKTL...,test
4,O00401,"N-WASP, WASL",RETSKVIYDFIEKTG,ABL1,MKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAV...,247,1,"[[[0.993, 0.965, 1.078, 1.0, 0.886, 1.229, 1.2...",RETSKVIYDFIEKTG_MKHKLGGGQYGEVYEGVWKKYSLTVAVKTL...,test
21,O14745,"NHERF1, SLC9A3R1",PALVRSASSDTSEEL,GRK6,RQYRVLGKGGFGEVCACQVRATGKMYACKKLEKKRIKKRKGEAMAL...,261,1,"[[[0.7417, 1.137, 1.097, 1.035, 0.98, 0.881, 1...",PALVRSASSDTSEEL_RQYRVLGKGGFGEVCACQVRATGKMYACKK...,test
36,O14920,"IKBKB, IKKB",AKELDQGSLCTSFVG,CHUK,MRERLGTGGFGNVCLYQHRELDLKIAIKSCRLELSTKNRERWCHEI...,272,1,"[[[1.23, 0.886, 0.965, 1.123, 0.873, 1.042, 1....",AKELDQGSLCTSFVG_MRERLGTGGFGNVCLYQHRELDLKIAIKSC...,test
36,O14920,"IKBKB, IKKB",DQGSLCTSFVGTLQY,CHUK,MRERLGTGGFGNVCLYQHRELDLKIAIKSCRLELSTKNRERWCHEI...,272,1,"[[[0.873, 1.042, 1.034, 1.0, 1.123, 1.449, 1.0...",DQGSLCTSFVGTLQY_MRERLGTGGFGNVCLYQHRELDLKIAIKSC...,test
...,...,...,...,...,...,...,...,...,...,...
2269,Q15256,PTPRR,PYQYIYFYRYEYKYE,PRKACA,FERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQIEHT...,254,0,"[[[0.87, 1.211, 0.9473, 1.211, 1.135, 1.211, 1...",PYQYIYFYRYEYKYE_FERIKTLGTGSFGRVMLVKHKETGNHYAMK...,test
2342,Q9Y5B0,CTDP1,IYGYQYLYGYLYGYR,CSNK2A2,YQLVRKLGRGKYSEVFEAINITNNERVVVKILKPVKKKKIKREVKI...,285,0,"[[[1.1875, 1.269, 0.981, 1.269, 1.003, 1.269, ...",IYGYQYLYGYLYGYR_YQLVRKLGRGKYSEVFEAINITNNERVVVK...,test
2342,Q9Y5B0,CTDP1,VSVSGSLSFSFSVSF,CSNK2A2,YQLVRKLGRGKYSEVFEAINITNNERVVVKILKPVKKKKIKREVKI...,285,0,"[[[1.165, 1.023, 1.165, 1.023, 0.981, 1.023, 1...",VSVSGSLSFSFSVSF_YQLVRKLGRGKYSEVFEAINITNNERVVVK...,test
2365,O60825,PFKFB2,GYGYVYRYGYVYEYQ,PRKAA2,VLGDTLGVGTFGKVKIGEHQLTGHKVAVKILNRQKIRSLDVVGKIK...,251,0,"[[[1.037, 1.165, 1.037, 1.165, 1.498, 1.165, 1...",GYGYVYRYGYVYEYQ_VLGDTLGVGTFGKVKIGEHQLTGHKVAVKI...,test


In [15]:
df = pd.concat([df_train, df_test])
df.head(3)

,sub_ID,sub_name,sub_seq,kin_name,kin_seq,len,answer,matrix,subseq_kinseq,df
7,Q9UQL6,HDAC5,FPLRKTASEPNLKVR,PKCD,HKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVE...,251,1,"[[[1.061, 0.825, 1.012, 0.963, 0.6885, 1.026, ...",FPLRKTASEPNLKVR_HKVLGKGSFGKVLLGELKGRGEYFAIKALK...,train
8,P18433-2,PTPRA iso2,PLLARSPSTNRKYPP,PKCD,HKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVE...,251,1,"[[[0.825, 1.012, 1.012, 1.032, 0.963, 0.9688, ...",PLLARSPSTNRKYPP_HKVLGKGSFGKVLLGELKGRGEYFAIKALK...,train
9,P10415,Bcl-2,RDPVARTSPLQTPAA,PKCD,HKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVE...,251,1,"[[[0.963, 0.9204, 0.825, 1.01, 1.032, 0.963, 1...",RDPVARTSPLQTPAA_HKVLGKGSFGKVLLGELKGRGEYFAIKALK...,train


In [17]:
import torch

In [20]:
for i in range(len(df)):
    df.iloc[i,-3] = torch.Tensor(df.iloc[i,-3])

In [23]:
dff = df[df.duplicated(['subseq_kinseq'])]
dff

,sub_ID,sub_name,sub_seq,kin_name,kin_seq,len,answer,matrix,subseq_kinseq,df
162,Q96D96,HVCN1,SKFLRHFTVVGDDYH,PKCD,HKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVE...,251,1,"[[[tensor(0.9688), tensor(0.6885), tensor(1.06...",SKFLRHFTVVGDDYH_HKVLGKGSFGKVLLGELKGRGEYFAIKALK...,train
176,Q96D96,HVCN1,GMLRKLFSSHRFQVI,PKCD,HKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVE...,251,1,"[[[tensor(1.0059), tensor(1.0820), tensor(1.01...",GMLRKLFSSHRFQVI_HKVLGKGSFGKVLLGELKGRGEYFAIKALK...,train
523,Q6NXS1,PPP1R2P3,YRIQEQESSGEEDSD,CK2A1,YQLVRKLGRGKYSEVFEAINITNNEKVVVKILKPVKKKKIKREIKI...,285,1,"[[[tensor(1.2686), tensor(1.0615), tensor(1.18...",YRIQEQESSGEEDSD_YQLVRKLGRGKYSEVFEAINITNNEKVVVK...,train
643,Q6NXS1,PPP1R2P3,RIQEQESSGEEDSDL,CK2A1,YQLVRKLGRGKYSEVFEAINITNNEKVVVKILKPVKKKKIKREIKI...,285,1,"[[[tensor(1.0615), tensor(1.1875), tensor(1.00...",RIQEQESSGEEDSDL_YQLVRKLGRGKYSEVFEAINITNNEKVVVK...,train
1339,P60891,PRPS1,GGAKRVTSIADRLNV,AMPKA1,LGDTLGVGTFGKVKVGKHELTGHKVAVKILNRQKIRSLDVVGKIRR...,250,1,"[[[tensor(0.9121), tensor(0.9121), tensor(1.30...",GGAKRVTSIADRLNV_LGDTLGVGTFGKVKVGKHELTGHKVAVKIL...,train
...,...,...,...,...,...,...,...,...,...,...
1290,P41743,"PKC-lambda, PRKCI",RLFFVIEYVNGGDLM,SRC,EVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMK...,242,1,"[[[tensor(1.1699), tensor(0.9839), tensor(0.87...",RLFFVIEYVNGGDLM_EVKLGQGCFGEVWMGTWNGTTRVAIKTLKP...,test
1771,Q05655,"PKC-delta, PRKCD",FGESRASTFCGTPDY,PDPK1,KFGKILGEGSFSTVVLARELATSREYAIKILEKRHIIKENKVPYVT...,247,1,"[[[tensor(0.9761), tensor(0.8359), tensor(1.19...",FGESRASTFCGTPDY_KFGKILGEGSFSTVVLARELATSREYAIKI...,test
1788,Q08379,"GM130, GOLGA2",REYQQRNSPGVPTGA,CDK1,YTKIEKIGEGTYGVVYKGRHKTTGQVVAMKKIRLESEEEGVPSTAI...,283,1,"[[[tensor(1.0615), tensor(0.8652), tensor(1.26...",REYQQRNSPGVPTGA_YTKIEKIGEGTYGVVYKGRHKTTGQVVAMK...,test
1959,Q15418,"RPS6KA1, p90RSK1",DHEKKAYSFCGTVEY,PDPK1,KFGKILGEGSFSTVVLARELATSREYAIKILEKRHIIKENKVPYVT...,247,1,"[[[tensor(1.0059), tensor(0.6885), tensor(1.19...",DHEKKAYSFCGTVEY_KFGKILGEGSFSTVVLARELATSREYAIKI...,test


In [27]:
df_train_dupl = dff[dff['df'] == 'test']
df_train_dupl

,sub_ID,sub_name,sub_seq,kin_name,kin_seq,len,answer,matrix,subseq_kinseq,df
21,O14745,"NHERF1, SLC9A3R1",PALVRSASSDTSEEL,GRK6,RQYRVLGKGGFGEVCACQVRATGKMYACKKLEKKRIKKRKGEAMAL...,261,1,"[[[tensor(0.7417), tensor(1.1367), tensor(1.09...",PALVRSASSDTSEEL_RQYRVLGKGGFGEVCACQVRATGKMYACKK...,test
62,O15111,"CHUK, IKKA",DQGSLCTSFVGTLQY,PRKCQ,HKMLGKGSFGKVFLAEFKKTNQFFAIKALKKDVVLMDDDVECTMVE...,251,1,"[[[tensor(0.9204), tensor(0.8862), tensor(1.00...",DQGSLCTSFVGTLQY_HKMLGKGSFGKVFLAEFKKTNQFFAIKALK...,test
455,P07948,LYN,RVIEDNEYTAREGAK,LYN,LVKRLGAGQFGEVWMGYYNNSTKVAVKTLKPGTMSVQAFLEEANLM...,243,1,"[[[tensor(1.0967), tensor(1.2383), tensor(1.19...",RVIEDNEYTAREGAK_LVKRLGAGQFGEVWMGYYNNSTKVAVKTLK...,test
1191,P35612,"ADD2, beta-adducin",KKKFRTPSFLKKSKK,PRKCD,HKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVE...,251,1,"[[[tensor(0.6885), tensor(0.6885), tensor(0.68...",KKKFRTPSFLKKSKK_HKVLGKGSFGKVLLGELKGRGEYFAIKALK...,test
1283,P40763,STAT3,DPGSAAPYLKTKFIC,SRC,EVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMK...,242,1,"[[[tensor(0.6729), tensor(0.6519), tensor(0.78...",DPGSAAPYLKTKFIC_EVKLGQGCFGEVWMGTWNGTTRVAIKTLKP...,test
1290,P41743,"PKC-lambda, PRKCI",RVIGRGSYAKVLLVR,SRC,EVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMK...,242,1,"[[[tensor(1.1699), tensor(0.9087), tensor(0.95...",RVIGRGSYAKVLLVR_EVKLGQGCFGEVWMGTWNGTTRVAIKTLKP...,test
1290,P41743,"PKC-lambda, PRKCI",LKKTDRIYAMKVVKK,SRC,EVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMK...,242,1,"[[[tensor(0.9839), tensor(1.1992), tensor(1.19...",LKKTDRIYAMKVVKK_EVKLGQGCFGEVWMGTWNGTTRVAIKTLKP...,test
1290,P41743,"PKC-lambda, PRKCI",RLFFVIEYVNGGDLM,SRC,EVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMK...,242,1,"[[[tensor(1.1699), tensor(0.9839), tensor(0.87...",RLFFVIEYVNGGDLM_EVKLGQGCFGEVWMGTWNGTTRVAIKTLKP...,test
1771,Q05655,"PKC-delta, PRKCD",FGESRASTFCGTPDY,PDPK1,KFGKILGEGSFSTVVLARELATSREYAIKILEKRHIIKENKVPYVT...,247,1,"[[[tensor(0.9761), tensor(0.8359), tensor(1.19...",FGESRASTFCGTPDY_KFGKILGEGSFSTVVLARELATSREYAIKI...,test
1788,Q08379,"GM130, GOLGA2",REYQQRNSPGVPTGA,CDK1,YTKIEKIGEGTYGVVYKGRHKTTGQVVAMKKIRLESEEEGVPSTAI...,283,1,"[[[tensor(1.0615), tensor(0.8652), tensor(1.26...",REYQQRNSPGVPTGA_YTKIEKIGEGTYGVVYKGRHKTTGQVVAMK...,test


In [13]:

tr_matrix = df_train.loc[:,['matrix']]
display(tr_matrix)

# list_train_matrix = list(df_train_matrix)
list_tr_matrix = tr_matrix.values.tolist()

,matrix
7,"[[[1.061, 0.825, 1.012, 0.963, 0.6885, 1.026, ..."
8,"[[[0.825, 1.012, 1.012, 1.032, 0.963, 0.9688, ..."
9,"[[[0.963, 0.9204, 0.825, 1.01, 1.032, 0.963, 1..."
10,"[[[1.006, 0.963, 1.006, 1.006, 0.963, 1.006, 1..."
11,"[[[1.01, 1.032, 1.061, 0.9688, 1.01, 0.9204, 0..."
...,...
35437,"[[[1.023, 1.027, 1.013, 1.027, 0.981, 1.027, 1..."
35438,"[[[1.124, 1.269, 1.1875, 1.269, 1.062, 1.269, ..."
35439,"[[[0.981, 1.269, 1.023, 1.269, 1.063, 1.269, 0..."
35440,"[[[1.003, 1.023, 1.027, 1.023, 1.003, 1.023, 1..."


In [ ]:
# duple_matrix.loc[:,['train_subkin_seq']] = 2
# duple_matrix.loc[:,['test_subkin_seq']] = 2
# # x : train / y : test
# duple_matrix['train_subkin_seq'] = duple_matrix['Substrate_seq_x'].str.cat(duple_matrix['Kinase_seq_x'], sep='_')
# duple_matrix['test_subkin_seq'] = duple_matrix['Substrate_seq_y'].str.cat(duple_matrix['Kinase_seq_y'], sep='_')


In [66]:
df_train.iloc[1,-2]

tensor([[[0.8252, 1.0117, 1.0117,  ..., 1.0635, 0.8252, 0.8252],
         [0.6089, 0.9580, 0.9580,  ..., 1.0127, 0.6089, 0.6089],
         [0.8213, 1.2383, 1.2383,  ..., 1.1650, 0.8213, 0.8213],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]])

In [54]:
for i in range(len(df_train)):
    df_train.iloc[i,-2] = torch.Tensor(df_train.iloc[i,-2])

In [56]:
for i in range(len(df_test)):
    df_test.iloc[i,-3] = torch.Tensor(df_test.iloc[i,-3])

In [59]:
# duple_matrix.loc[:,['seq_duple_comparison']] = 2

df_test.loc[:,['is_in_train?']] = 10

for i in range(len(df_test)):
    if df_test.iloc[i,-3] in df_train.iloc[:,-2]:
        df_test.iloc[i,-1] = 1
    else:
        df_test.iloc[i,-1] = 0
        
# df_test

In [62]:
df_test.iloc[1,-3] == df_train.iloc[1,-2]

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True]]])

In [ ]:
duple_matrix

import pandas as pd
  # CSV파일로 저장하기
duple_matrix.to_csv("/home/hb/python/git_pp_pred/data/220502_pp-pred_(train-test_dt-duplicate-comparison-by-matrix-&seq)_f1_HB.csv")


import pandas as pd
  # 엑셀로 저장하기
duple_matrix.to_excel("/home/hb/python/git_pp_pred/data/220502_pp-pred_(train-test_dt-duplicate-comparison-by-matrix-&seq)_f1_HB.xlsx")

  # 엑셀파일 읽기
  

In [ ]:
for i in range(len(duple_matrix)):
    if duple_matrix.iloc[i,-3] == duple_matrix.iloc[i,-2]:
        duple_matrix.iloc[i,-1] = 1
    else:
        duple_matrix.iloc[i,-1] = 0




In [ ]:
traindata_true = traindata[traindata['answer'] == 1]
traindata_false = traindata[traindata['answer'] == 0]

testdata_true = testdata[testdata['answer'] == 1]
testdata_false = testdata[testdata['answer'] == 0]

df_traintrue_matrix = traindata_true.iloc[:,-1]
df_trainfalse_matrix = traindata_false.iloc[:,-1]

list_traintrue_matrix = df_traintrue_matrix.values.tolist()
list_trainfalse_matrix = df_trainfalse_matrix.values.tolist()

testdata_true.loc[:,['train_matrix']] = 2

for i in range(len(testdata_true)):
    if testdata_true.iloc[i,-2] in list_traintrue_matrix:
        testdata_true.iloc[i,-1] = 1
    else:
        testdata_true.iloc[i,-1] = 0
        
testdata_true.head(10)
        
testdata_false.loc[:,['train_matrix']] = 2

for i in range(len(testdata_false)):
    if testdata_false.iloc[i,-2] in list_trainfalse_matrix:
        testdata_false.iloc[i,-1] = 1
    else:
        testdata_false.iloc[i,-1] = 0
        
testdata_false.head(10)
        

# 트레인과 중복되는 테스트 데이터
testdata_true_anwswer1 = testdata_true[testdata_true['train_matrix'] == 1]
display(testdata_true_anwswer1.head(5))
print(len(testdata_true_anwswer1))
# print(len(testdata_true))

# # 트레인과 중복되지 않는 테스트 데이터
# testdata_true_anwswer0 = testdata_true[testdata_true['train'] == 0]
# display(testdata_true_anwswer0.head(5))
# print(len(testdata_true_anwswer0))
# print(len(testdata_true))


In [ ]:
train_test_duplicate_comparison = testdata_true_anwswer1
len(train_test_duplicate_comparison)

In [ ]:
import pandas as pd
  # CSV파일로 저장하기
train_test_duplicate_comparison.to_csv("/home/hb/python/git_pp_pred/data/220502_pp-pred_(train-test_true-dt_duplicate-comparison)_f1_HB.csv")


In [ ]:
traindata

In [ ]:
train = traindata.iloc[:,[2,3,4]]
test = testdata.iloc[:,[2,3,4]]
display(train.head(3))
display(test.head(3))

In [ ]:
traindata

In [ ]:
def subseq_kinseq(df):
    df.loc[:,'subseq_kinseq'] = 0
    df['subseq_kinseq'] = df['Substrate_seq'].str.cat(df['Kinase_seq'], sep='_')


subseq_kinseq(traindata)
subseq_kinseq(testdata)
subseq_kinseq(train_test_duplicate_comparison)

In [ ]:
traindata

In [ ]:
traindata_true = traindata[traindata['answer'] == 1]
traindata_false = traindata[traindata['answer'] == 0]


df_traintrue_matrix = traindata_true.iloc[:,-1]
df_trainfalse_matrix = traindata_false.iloc[:,-1]

list_traintrue_matrix = df_traintrue_matrix.values.tolist()
list_trainfalse_matrix = df_trainfalse_matrix.values.tolist()

testdata_true.loc[:,['train']] = 2

for i in range(len(testdata_true)):
    if testdata_true.iloc[i,-2] in list_traintrue_matrix:
        testdata_true.iloc[i,-1] = 1
    else:
        testdata_true.iloc[i,-1] = 0
        
testdata_true.head(10)
        
testdata_false.loc[:,['train']] = 2

for i in range(len(testdata_false)):
    if testdata_false.iloc[i,-2] in list_trainfalse_matrix:
        testdata_false.iloc[i,-1] = 1
    else:
        testdata_false.iloc[i,-1] = 0
        
testdata_false.head(10)
        

# 트레인과 중복되는 테스트 데이터
testdata_true_anwswer1 = testdata_true[testdata_true['train'] == 1]
display(testdata_true_anwswer1.head(5))
print(len(testdata_true_anwswer1))
# print(len(testdata_true))

# # 트레인과 중복되지 않는 테스트 데이터
# testdata_true_anwswer0 = testdata_true[testdata_true['train'] == 0]
# display(testdata_true_anwswer0.head(5))
# print(len(testdata_true_anwswer0))
# print(len(testdata_true))


In [ ]:
duple = pd.merge(train, test, left_on='subseq_kinseq', right_on = 'subseq_kinseq', how='inner')
duple # train, test 비교 시 중복되는 subseq_kinseq 리스트 : 개수 50개

duple_df = duple.loc[:,['subseq_kinseq']]

In [ ]:
duple_df

In [ ]:
duple_list = duple_df.values.tolist()
duple_list[0]

In [ ]:
train_test_duplicate_comparison

In [ ]:
train_test_duplicate_comparison.loc[:,'seq_vs_matrix'] = 2
        
duplelist = list(duple.loc[:,["subseq_kinseq"]])
duplelist

for i in range(len(train_test_duplicate_comparison)):
    if train_test_duplicate_comparison.iloc[i,-2] in duple_list:
        train_test_duplicate_comparison.iloc[i,-1] = 1
    else:
        train_test_duplicate_comparison.iloc[i,-1] = 0
        
train_test_duplicate_comparison


# # 트레인과 중복되는 테스트 데이터
train_test_duplicate_comparison_seqmatrix_1 = train_test_duplicate_comparison[train_test_duplicate_comparison['seq_vs_matrix'] == 1]
train_test_duplicate_comparison_seqmatrix_1

In [ ]:
train_test_duplicate_comparison